In [ ]:
%load_ext autoreload

# 引入必要的库

In [ ]:
%autoreload
import sys,os
import numpy as np
import matplotlib.pyplot as plt
from tianmoucv.isp import SD2XY
import torch
from tianmoucv.data import TianmoucDataReader
import torch.nn.functional as F
import cv2

train = '/data/lyh/tianmoucData/tianmoucReconDataset/train/normal'
all_data = [train]
key_list = []
print('---------------------------------------------------')
for sampleset in all_data:
    print('---->',sampleset,'有：',len(os.listdir(sampleset)),'个样本')
    for e in os.listdir(sampleset):
        print(e,end=',')
        key_list.append(e)


# 简单观察若干帧图像并输出进行观察(单目)

- TianmoucDataReader注释(0.3.5版本)
    - ## [输入]
    - 输入dataPath：该路径下应当包含1个或多个子目录，每个子目录对应1段Tianmouc视频。
        - 支持string格式(仅输入1个地址)或list格式(输入1个或多个地址)。
           - 这个地址可以是一个tmdat sample的绝对路径，也可以是数据集的路径
           - 如果是数据集路径(path或者path的任意多层子文件夹内有多个tmdat),可以用matchkey读取特定sample，也可以合并读取
        - 对于单目数据，每个sample下应包含rod和cone两个目录，多目数据额外还有目录rod_N和cone_N，N为相机编号N>=1
        - 双目数据补充：20240201测试结果，在实验过程中重启GUI不会导致两个相机标签交换
            - 只要不插拔并交换接线，整个数据集中相机的idx将保持不变
    - 输入N：返回的dataset中包含多个sample，每个sample包含(N+1)帧COP，以及中间的所有AOP帧。
        - 默认N=1，在757fps模式下sample中有F0，F1两帧COP，以及中间的(25+1)帧AOP，最后一帧AOP与下一个sample第1帧AOP相同，可以跳过。
    - 输入matchkey：在dataPath所有路径下的子目录名称中匹配对应的，否则会输出所有数据。
        - 若输入超过1个路径，建议不同路径下不要出现同名子目录，否则可能出现bug。
    - 输入camera_idx：默认为0，表示识别单目输入，若为双目数据，则camera_idx=0,1分别录取双目数据。
    - 原先版本中的输入参数MAXLEN强制默认设为-1，即始终为读取全部数据。
    - ## [输出]
    - 输出dataset调用方式类似于列表，通过sample = dataset[index]逐一获取数据。
    - sample为字典类型，包含如下key
        - COP帧依次记录为F0，F1，F2...F(N)
            - COP的精确帧率为30.3fps
            - 'F0'默认使用ISP算法调色, 可以关闭
            - 'F0_without_isp'不加额外处理，若加红外滤光片应使用这个数据
            - 'F0_HDR'为简易融合算法处理结果，由同步的SD和RGB合成高动态图
        - AOP帧
            - 'rawDiff'为AOP像素原始输出(160×160), 为tianmuocv非神经网络预处理接口的输入
            - 'tsdiff'为rawDiff直接插值得到的与COP同分辨率的图像(320×640), 用于神经网络的输入
            - 上述三个对应的key_value均为张量格式，torch.Size([3, X, height, width])
                - 第0个维度为3，分别依次对应TD，SD1，SD2
                - 第1个维度对应AOP帧数目，在757fps模式下X=N×25+1, 每25帧为一个单位
                - 第2，3个维度对应AOP帧的分辨率
        - 'sysTimeStamp'为系统初始时间，用于在多目相机情况下进行时间对齐。
            - 两相机之间初始时间差为sysTimeStamp1-sysTimeStamp2，单位为秒
            - COP对齐时若Δt>33ms/2，建议让相机1的第K+Δt/33ms帧COP与相机2的第K帧COP对齐，这样时间差更小。
        - 'labels'用于标注HDR，HS，Blur，Noisy等4种极端情况分类，暂未实装。
        - 'meta'包含了该段目录的一些元数据，如文件存储目录，时间戳等等，需要详细数据分析时使用
        

## (测试功能)封装读出的数据

In [ ]:
%autoreload
import sys,os
import numpy as np
import torch
import torch.nn.functional as F
import cv2
import math,time
import matplotlib.pyplot as plt

from tianmoucv.isp import SD2XY

aim = 4 # 读取的目标COP帧的帧号
key_list = ['outdoor_cross_3']
datapath = all_data
N = 2
for key in key_list:
    dataset = TianmoucDataReader(datapath,N=N,matchkey=key,camera_idx=0,use_data_parser = True)
    img_list = []
    for index in range(aim,min(aim+1,len(dataset))):
        sample = dataset[index]
        sample.help()
        dataRatio = sample.get_data_rate()
        plt.figure(figsize=(12,2*N))  
        for i in range(N):
            F = sample.get_rgb(i)
            F_HDR = sample.get_hdr_fusion(i)
            tsdiff = torch.Tensor(sample.get_tsd(i*25,ifUpSampled=True))
            plt.subplot(N,3,1+i*3)
            plt.imshow(F)
            plt.subplot(N,3,2+i*3)
            plt.imshow(tsdiff[:,0,...].permute(1,2,0)*16) #可视化第0帧，乘个Gain方便看
            plt.subplot(N,3,3+i*3)
            plt.imshow(F_HDR)
            
        plt.show()
    break